In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import qutip
from copy import deepcopy
from CoupledQuantumSystems.systems import FluxoniumOscillatorSystem
from CoupledQuantumSystems.drive import DriveTerm,square_pulse_with_rise_fall
from CoupledQuantumSystems.evo_parrallel import run_parallel_ODEsolve_and_post_process_jobs_with_different_systems

In [2]:
max_ql = 25
max_ol = 200
EJ = 4
EC = EJ/2.0
EL = EJ/30
Er = 13.63285461

g = 0.3
w_d = 13.6355
amp = 0.01
kappa = 2e-3

tot_time =500
tlist = np.linspace(0, tot_time, tot_time)
# system  =  FluxoniumOscillatorSystem(
#                 EJ = EJ,
#                 EC = EC,
#                 EL = EL,
#                 Er = Er,
#                 g_strength = g,
#                 kappa=kappa,
#                 qubit_level = max_ql,
#                 osc_level = max_ol,
#                 products_to_keep=[[ql, ol] for ql in range(max_ql) for ol in range(max_ol) ],
#                 computaional_states = '0,2',
#                 )

# with open('gf_mcsolve_system_g3a01k002.pkl', 'wb') as f:
#     pickle.dump(system,f)

In [3]:

with open('gf_mcsolve_system_g3a01k002.pkl', 'rb') as f:
    system = pickle.load(f)

# 2 store the mcsolve jobs

In [4]:
# state_leak_dressed = qutip.basis(max_ql * max_ol, system.product_to_dressed[(1,0)])
state_0_dressed = qutip.basis(max_ql * max_ol, system.product_to_dressed[(0,0)])
state_1_dressed = qutip.basis(max_ql * max_ol, system.product_to_dressed[(2,0)])
state_plus_X_dressed = (state_0_dressed +  state_1_dressed).unit()
state_minus_X_dressed = (state_0_dressed - state_1_dressed).unit()
state_plus_Y_dressed = (state_0_dressed + 1j * state_1_dressed).unit()
state_minus_Y_dressed = (state_0_dressed - 1j * state_1_dressed).unit()

initial_states  = [
                    # state_leak_dressed,
                   state_0_dressed,
                   state_1_dressed,
                   state_plus_X_dressed,
                   state_minus_X_dressed,
                   state_plus_Y_dressed,
                   state_minus_Y_dressed
                   ]

# leakage_products_to_keep = [[ql, ol] for ql in [1] for ol in range(max_ol) ]
computational_products_to_keep = [[ql, ol] for ql in range(10) for ol in range(100) ]
list_of_products_to_keep = [
    # leakage_products_to_keep,
    computational_products_to_keep,
    computational_products_to_keep,
    computational_products_to_keep,
    computational_products_to_keep,
    computational_products_to_keep,
    computational_products_to_keep
]

list_of_systems = []
list_of_kwargs = []
for products_to_keep, y0 in zip(list_of_products_to_keep, initial_states):
    system.set_new_product_to_keep(products_to_keep)
    system.set_new_operators_after_setting_new_product_to_keep()
    list_of_systems.append(deepcopy(system))
    list_of_kwargs.append( {
        'y0':system.truncate_function(y0) ,
        'tlist':tlist,
        'drive_terms':[DriveTerm( 
                            driven_op= system.driven_operator,
                            pulse_shape_func=square_pulse_with_rise_fall,
                            pulse_shape_args={
                                'w_d': w_d ,
                                'amp': amp,
                                't_rise': 20,
                                't_square': tot_time
                            })],
        'e_ops':[system.a_trunc , system.a_trunc.dag()*system.a_trunc],
        # 'c_ops':[kappa *qutip.lindblad_dissipator(system.a_trunc) ]
        })
    

results = run_parallel_ODEsolve_and_post_process_jobs_with_different_systems(
    list_of_systems,
    list_of_kwargs,
    post_processing = ['pad_back']
)

import pickle
with open('sesolve_large_dim_a01_tomo.pkl', 'wb') as file:
    pickle.dump(results, file)


 Total run time: 4584.00s] Elapsed 4584.00s / Remaining 00:00:00:00


Processing states with pad_back_custom:   9%|▉         | 47/500 [00:01<00:18, 24.54it/s]

 [*********97%********** ] Elapsed 4585.87s / Remaining 00:00:02:21

Processing states with pad_back_custom:  14%|█▍        | 71/500 [00:02<00:18, 23.04it/s]

 [*********97%********** ] Elapsed 4587.02s / Remaining 00:00:02:21

...

Processing states with pad_back_custom:  27%|██▋       | 133/500 [00:04<00:12, 28.67it/s]

 Total run time: 4717.85s] Elapsed 4717.85s / Remaining 00:00:00:00


Processing states with pad_back_custom:   0%|          | 0/500 [00:00<?, ?it/s]26.53it/s]

 Total run time: 4723.92s] Elapsed 4723.92s / Remaining 00:00:00:00


Processing states with pad_back_custom: 100%|██████████| 500/500 [00:17<00:00, 28.79it/s]


In [ ]:
import pickle
with open('sesolve_large_dim_a01_tomo.pkl', 'rb') as file:
    results = pickle.load(file)
